# Downloading packages

In [ ]:
!pip3 install pyspellchecker
!nltk.download('wordnet')

# Imports

In [ ]:
import re
import spacy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from os import cpu_count
from multiprocessing import Process, Queue
from spellchecker import SpellChecker
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

%matplotlib inline

# Global Variables

In [ ]:
N_PROCESSES = cpu_count()
STEMMER = WordNetLemmatizer()
SPELL_CHECKER = SpellChecker()
STOP_WORDS = set(stopwords.words("english"))

# Utility Functions

In [ ]:
def cal_skewness(df, cols, threshold=1):
    skewed_cols = []
    for col in cols:
        if abs(df[col].skew()) >= threshold:
            skewed_cols.append(col)
            
    return skewed_cols

In [ ]:
def preprocess_text(text):
    # Removing URLs
    url = re.compile(r"https?://\S+|www\.\S+")
    text = url.sub("", text)
    
    # Removing HTML tags & contents inside it
    html=re.compile("<.*?>.*</?.*?>")
    text = html.sub("", text)
    
    # Removing non-alpha characters
    punct = re.compile(r"[^a-zA-Z\s']")
    text = punct.sub("", text)
    
    # Remove extra white spaces
    extra_white_spaces = re.compile("\s{2,}")
    text = extra_white_spaces.sub(" ", text)
    
    # Correcting misspelled words
    splitted_text = text.split()
    misspelled_words = set(SPELL_CHECKER.unknown(splitted_text))
    if len(misspelled_words) > 0:
        corrected_text = []
        for word in splitted_text:
            if word in misspelled_words:
                corrected_text.append(SPELL_CHECKER.correction(word))
            else:
                corrected_text.append(word)
        text = " ".join(corrected_text)

    return text

In [ ]:
class PreprocessConcurrent:
    def __init__(self, chunk_size=100):
        self.chunk_size = chunk_size
    
    def preprocess_texts(self, text_series):
        start_chunk_idx, end_chunk = 0, text_series.shape[0]
        processes, q = list(), Queue()
        while start_chunk_idx < end_chunk:
            last_chunk_start_idx, cur_process_idx = start_chunk_idx, 1
            while cur_process_idx <= N_PROCESSES and start_chunk_idx < end_chunk:
                if start_chunk_idx+self.chunk_size <= end_chunk:
                    p = Process(target=self._preprocess_text_parallel, args=(
                        text_series[start_chunk_idx:start_chunk_idx+self.chunk_size], q
                    ))
                else:
                    p = Process(target=self._preprocess_text_parallel, args=(
                        text_series[start_chunk_idx:end_chunk], q
                    ))
                p.start()
                processes.append(p)
                cur_process_idx += 1
                start_chunk_idx += self.chunk_size
            
            print(f"Currently processing chunks from {last_chunk_start_idx} to {start_chunk_idx} "
                  f"out of total {end_chunk} chunks")
            while len(processes) > 0:
                p = processes.pop()
                p.join()

            while not q.empty():
                start, end, preprocessed_text_series = q.get()
                text_series[start:end].update(preprocessed_text_series)
                
        q.close()
    
    def _preprocess_text_parallel(self, texts, q):
        for idx, text in texts.items():
            texts[idx] = preprocess_text(text)

        q.put([texts.index[0], texts.index[-1], texts])

In [ ]:
def tokenize_text(text):
    return([STEMMER.lemmatize(token) for token in text.lower().split() if token not in STOP_WORDS])

# Preparing dataset & EDA

In [ ]:
train_df = pd.read_csv("../input/nlp-getting-started/train.csv")
test_df = pd.read_csv("../input/nlp-getting-started/test.csv")
train_labels = train_df["target"]
train_df.drop(columns=["target"], inplace=True)
print(f"Training shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")

# Feature generations

In [ ]:
concurrency = PreprocessConcurrent()
print("Original:\n", train_df.text[10:20])
concurrency.preprocess_texts(train_df.text)
print("Processed:\n", train_df.text[10:20])

In [ ]:
vectorizer = TfidfVectorizer(
    tokenizer=tokenize_text, max_features=1000, strip_accents="ascii", 
)
X = vectorizer.fit_transform(train_df.text).todense()

# Model Training

In [ ]:
# Gaussian Naive Bayes
gaussian_nb = GaussianNB()
scores = cross_val_score(gaussian_nb, X, train_labels, scoring="f1")
plt.plot(scores)
plt.show()

In [ ]:
# Logistic Regression
param_grid = {"penalty": ["l2"], "C": [0.01, 1, 10], "max_iter": [100, 400, 800], "solver": ["newton-cg"]}
logistic_reg = GridSearchCV(LogisticRegression(), param_grid, scoring="f1")
logistic_reg.fit(X, train_labels)
print(f"Best score: {logistic_reg.best_score_}")
print(f"Best parameters: {logistic_reg.best_estimator_}")

# Testing

In [ ]:
print("Original:\n", test_df.text[10:20])
concurrency.preprocess_texts(test_df.text)
print("Processed:\n", test_df.text[10:20])
X_test = vectorizer.transform(test_df.text).todense()

In [ ]:
gaussian_nb.fit(X, train_labels)
submission = pd.read_csv("../input/nlp-getting-started/sample_submission.csv")
submission["target"] = gaussian_nb.predict(X_test)
submission.to_csv("result.csv", index=False)
submission.head()